# Vladimir Nikiforov

### PART 0 - PREPARE ENV

In [1]:
# Stop previously runned server
!docker stop ht_pg_server
!docker rm $(docker ps -a -q)
!docker rmi $(docker images -q)

ht_pg_server
29896e0c0fff
Untagged: postgres:11
Untagged: postgres@sha256:68b49a280d2fbe9330c0031970ebb72015e1272dfa25f0ed7557514f9e5ad7b7
Deleted: sha256:53912975086f1470f877922e3eb79c4f17c403fcb0b51b1f45df9b623987600e
Deleted: sha256:859135cc89a4161becb56082dc95772d26d157bc67622be28ccab159cee9c520
Deleted: sha256:85d5ba072f703aed8ed136ee24ef0a5c22fb818e307db40be0637df1691546fe
Deleted: sha256:44bc37a16317acf120ebcb0b8aa2cf8738f0dcc40ca47f18d9d5789099089cd0
Deleted: sha256:050e2fbc4c511fff109624929dcc24d61f4565052ffd6e30e3d46d42ce65c576
Deleted: sha256:6e7901ce413d382c6544cf07b5370341507a4734525947ecef28944c041e39b6
Deleted: sha256:c0d5a0e28851bc9a301c0cb92e03c036c1a4b0d8da1efc4a16a921abdd641e3e
Deleted: sha256:21cf04dd5b4e7ddbe4a7f122b532038c351eef9057ff71940a1a0a36069fabd5
Deleted: sha256:5eba02d7eb8edfd2d0c4503cd6976cfb526e96c4f62960bac5efdf86a1e71e4f
Deleted: sha256:0e0ff67afa7921baa6b3c7afd83664752663b5b8704c5236941359003e6f8887
Deleted: sha256:915e9e2c13848dc5933cea4c23b53649971

In [2]:
# Run the server:
!docker run -d --name ht_pg_server -v ht_dbdata:/var/lib/postgresql/data -p 54320:5432 postgres:11

Unable to find image 'postgres:11' locally
11: Pulling from library/postgres














Digest: sha256:68b49a280d2fbe9330c0031970ebb72015e1272dfa25f0ed7557514f9e5ad7b7
Status: Downloaded newer image for postgres:11
edb79af76f1c6cd62c0e974d6173d54e4c99643d8b37b0a6c4a89fddee0ac5ca


In [3]:
# Checking the logs to see if it is running:
!docker logs ht_pg_server

2019-08-04 12:53:49.761 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432
2019-08-04 12:53:49.761 UTC [1] LOG:  listening on IPv6 address "::", port 5432
2019-08-04 12:53:49.773 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
2019-08-04 12:53:49.792 UTC [30] LOG:  database system was shut down at 2019-08-04 12:53:17 UTC
2019-08-04 12:53:49.799 UTC [1] LOG:  database system is ready to accept connections


In [4]:
# Creating the database:
!docker exec -it ht_pg_server psql -U postgres -c "drop database ht_db"
!docker exec -it ht_pg_server psql -U postgres -c "create database ht_db"

DROP DATABASE
CREATE DATABASE


In [5]:
import os
# change working directory to the directory with code
os.chdir('../code/')

In [6]:
!cat etl.py

import psycopg2 as pg
import yaml
from pathlib import Path
import os
import pandas as pd

def create_tables(config, connection):
    cur = connection.cursor()
    for table in config:
        name = table.get('name')
        schema = table.get('schema')
        ddl = f"""CREATE TABLE IF NOT EXISTS {name} ({schema})"""
        cur.execute(ddl)

    connection.commit()
    cur.close()

def transform_tables(config):

    data_path = "../data/"

    for table in config:
        table_name = table.get('name')
        table_source = os.path.join(data_path,f"{table_name}.csv")
        table_cols = []
        for i in table.get('columns'):
            table_cols.append(str.upper(i))
        df = pd.read_csv(table_source)
        df_reorder = df[table_cols]  # rearrange column here
        df_reorder.to_csv(table_source, index=False)

def load_tables(config, connection):

    # iterate and load
    cur = connection.cursor()
    data_path = "../data/"

    for table in config:
        table_name

In [7]:
# execute ETL-script
!python etl.py

/home/truename/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [8]:
!cat features.py

# create features and store them in the DB

import psycopg2 as pg
import yaml
from pathlib import Path
import os
import pandas as pd
from datetime import datetime
import dateutil


def prepare_user_features(users_file_name):
    user_df = pd.read_csv(users_file_name)
    # There are NaN in TERMS_VERSION - fill it by "Undefined" version
    user_df['TERMS_VERSION'] = user_df['TERMS_VERSION'].fillna("Undefined")
    user_df.drop('STATE', axis=1, inplace=True)
    # Create AGE variable as age on CREATED_DATE date of profile in system
    user_df['AGE'] = user_df.apply(lambda x: datetime.strptime(x['CREATED_DATE'], '%Y-%m-%d %H:%M:%S.%f').year-x['BIRTH_YEAR'], axis=1)
    # We don't need BIRTH_YEAR anymore - drop it
    user_df.drop('BIRTH_YEAR', axis=1, inplace=True)
    return user_df


def prepare_transactions_features(user_df, transactions_file_name):
    trn_df = pd.read_csv(transactions_file_name)
    df = trn_df.drop('ID', axis=1).merge(user_df, left_on="USER_ID", right_on="ID").dro

In [9]:
# execute script to create features and store them to the DB
!python features.py

/home/truename/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
features.py:66: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if old_col in dataset['DAY_OF_TRANSACTION'].unique()
features.py:84: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if old_col in dataset['HOUR_OF_TRANSACTION'].unique()


In [10]:
!cat train.py

# generates the fitted model artifact

def load_datasets(dataset_file_name, dataset_time_file_name):
    # Load and merge prepared datasets to the final dataset
    
    import pandas as pd

    # We don't need CURRENCY as a feature because FRAUDSTER is not identified by CURRENCY
    df = pd.read_csv(dataset_file_name)
    df_time = pd.read_csv(dataset_time_file_name)

    # Let's create dummy columns for categorical variables
    df = pd.get_dummies(df, columns=['ENTRY_METHOD','TYPE','SOURCE','STATE','MERCHANT_CATEGORY','TERMS_VERSION','KYC'])
    # and collect final dataset with time-features from dataset_time
    df = df.merge(df_time, left_on="USER_ID", right_on="USER_ID")
    del df_time
    return df


def oversampling_dataset(df):
    # Do oversampling of unbalanced class
    
    import numpy as np
    # split dataset to features and predicted value
    X, y = df.drop('IS_FRAUDSTER',axis=1), df['IS_FRAUDSTER']

    # balancing classes
    classes_count = y.value_counts()
    ma

In [11]:
# execute script to train the model
!python train.py

/home/truename/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=De

In [12]:
!cat patrol.py

# Load and identify users by pretrained model

def get_user_data(user_id, model_features, cat_features):
    # Load data from DB for specific user_id, split cat_features to dummies and fill absented categorical features' values by zeros to create complete dataset with model_features
    import numpy as np
    import pandas as pd
    import psycopg2
    # Create connection to DB
    connection = psycopg2.connect(
        host='localhost',
        port=54320,
        dbname='ht_db',
        user='postgres',
    )
    # open cursor
    c = connection.cursor()
    # load dataset for specific user_id
    c.execute(f"SELECT * FROM dataset t1, dataset_time t2 where t1.USER_ID = t2.USER_ID and t1.USER_ID = '{user_id}';")
    # get all rows from opened cursor
    result = c.fetchall()
    # check that we have one or more rows
    assert len(result) > 0, f"We don't have data for user_id={user_id}!"
    # get column' names from cursor to create dataframe
    colnames = [desc[0].upper() for desc i

In [13]:
# import function "patrol" from script "patrol.py"
from patrol import patrol

In [14]:
# Check Not a fraudster
patrol(user_id='1872820f-e3ac-4c02-bdc7-727897b60043')

/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/truename/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


['PASS']

In [15]:
# Check Fraudster
patrol(user_id='fb23710b-609a-49bf-8a9a-be49c59ce6de')

['LOCK_USER']